In [3]:
from matplotlib import pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

plt.style.use('Solarize_Light2')

- **Load CPIAUCSL data**

In [4]:
CPIAUCSL_df = pd.read_csv('../data/CPIAUCSL.csv')

CPIAUCSL_df['DATE'] = pd.to_datetime(CPIAUCSL_df['DATE'])
CPIAUCSL_df.set_index('DATE', inplace=True)

CPIAUCSL_df['^ %'] = CPIAUCSL_df['CPIAUCSL'].ffill().pct_change()

CPIAUCSL_df.tail()

,CPIAUCSL,^ %
DATE,,
2025-08-01,323.364,0.003825
2025-09-01,324.368,0.003105
2025-10-01,NaN,0.000000
2025-11-01,325.031,0.002044
2025-12-01,326.030,0.003074


- **Average change**

In [5]:
print(f"Average change is {CPIAUCSL_df['^ %'].mean() * 100}%")

Average change is 0.28819906604609774%


- **Overview on change**

In [6]:
positive_filter = CPIAUCSL_df['^ %'] > 0
negative_filter = CPIAUCSL_df['^ %'] < 0
zero_filter = CPIAUCSL_df['^ %'] == 0

# count
total_items = len(CPIAUCSL_df)

# average change
avg_change = [
    CPIAUCSL_df[positive_filter]['^ %'].mean(),
    CPIAUCSL_df[negative_filter]['^ %'].mean(),
    0,
    ""
]

# frequency
frequency = [
    CPIAUCSL_df[positive_filter]['^ %'].count(),
    CPIAUCSL_df[negative_filter]['^ %'].count(),
    CPIAUCSL_df[zero_filter]['^ %'].count()
]
frequency.append(frequency[0]/frequency[1])

# frequency pct
frequency_pct = [100 * frequency[i]/total_items for i in range(3)]
frequency_pct.append("")

# Prob adj
prob_adj = [frequency_pct[i] * avg_change[i] for i in range(3)]
prob_adj.append("")

pd.DataFrame({
    "%": [x * 100 for x in avg_change],
    "Frequency": frequency,
    "Frequency %": frequency_pct,
    "Prob Adjust % Change": prob_adj,
}, index=["Av Pos", "Av Neg", "Zero", "Ratio P/N"])

,%,Frequency,Frequency %,Prob Adjust % Change
Av Pos,0.372492,796.000000,83.966245,0.312767
Av Neg,-0.21632,109.000000,11.49789,-0.024872
Zero,0,42.000000,4.43038,0.0
Ratio P/N,,7.302752,,


- **Stats**

In [7]:
from scipy.stats import describe

# Display the results

stats = describe(CPIAUCSL_df['^ %'].dropna().tolist())
pd.DataFrame(
    {
        'value': [
            str(stats.nobs),
            stats.minmax[0] * 100,
            stats.minmax[1] * 100,
            stats.mean * 100,
            CPIAUCSL_df['^ %'].median() * 100,
            CPIAUCSL_df['^ %'].mode(dropna=True)[0] * 100,
            stats.variance,
            stats.skewness,
            stats.kurtosis
        ]
    },
    index=['nobs', 'Min %', 'Max %', 'Mean %', "Median %", "Mode %", 'Variance', 'Skewness', 'Kurtosis'],
)

,value
nobs,947
Min %,-1.770548
Max %,1.964286
Mean %,0.288199
Median %,0.246914
Mode %,0.0
Variance,0.000012
Skewness,0.598224
Kurtosis,4.098528


- **Data preview**

In [8]:
# Define bins
bins = [0.005 * i for i in range(-3, 4)]
bins.append(stats.minmax[1])
bins.insert(0, stats.minmax[0])
bin_labels = [f"{bins[i] * 100}% to {bins[i+1] * 100}%" for i in range(len(bins) - 1)]
bin_labels[0] = f"Less than {bins[1] * 100}%"
bin_labels[-1] = f"Greater than {bins[-2] * 100}%"

# Assign data to bins
binned = pd.cut(CPIAUCSL_df['^ %'], bins=bins, labels=bin_labels, include_lowest=True)

# Calculate frequency, probability, and cumulative probability
frequency = binned.value_counts().sort_index()
probability = 100 * frequency / frequency.sum()
cumulative_probability = probability.cumsum()

occurrence_frequencies = pd.DataFrame({
    'Frequency': frequency.values,
    'Probability %': probability.values,
    'Cumulative Probability %': cumulative_probability.values
}, index=bin_labels)

occurrence_frequencies

,Frequency,Probability %,Cumulative Probability %
Less than -1.5%,1,0.105597,0.105597
-1.5% to -1.0%,0,0.000000,0.105597
-1.0% to -0.5%,9,0.950370,1.055966
-0.5% to 0.0%,141,14.889124,15.945090
0.0% to 0.5%,605,63.885956,79.831045
0.5% to 1.0%,155,16.367476,96.198522
1.0% to 1.5%,29,3.062302,99.260824
Greater than 1.5%,7,0.739176,100.000000


In [9]:
fig = px.bar(occurrence_frequencies, x=occurrence_frequencies.index, y='Probability %')

fig = px.bar(
    occurrence_frequencies,
    x=occurrence_frequencies.index,
    y='Probability %',
)

fig.update_layout(
    title=dict(text='CPIAUCSL % change'),
    width=800,
    height=600,
    yaxis=dict(title="Probability %"),
    hovermode="x unified",
    template="plotly_dark"
)

fig.show()

- CPIAUCSL pct change

In [10]:
fig = go.Figure()

# Reference line
fig.add_trace(go.Scatter(
    x=CPIAUCSL_df.index,
    y=[100 * stats.mean] * (stats.nobs + 1),
    name='Average % change',
    line=dict(
        width=2,
        color="#F2C14E",   # muted gold
        dash="dash"
    ),
    hoverinfo="skip",
    # showlegend=False,
))

# Bars
fig.add_trace(go.Bar(
    x=CPIAUCSL_df.index,
    y=CPIAUCSL_df['^ %'] * 100,
    name='CPIAUCSL',
    marker=dict(
        color="#4C78A8",  # professional blue
        line=dict(color="#4C78A8", width=0)
    ),
    width=0.5,
))

fig.update_layout(
    title=dict(
        text='CPIAUCSL pct change',
        x=0.02,
        font=dict(size=20)
    ),
    plot_bgcolor="#0E1117",
    paper_bgcolor="#0E1117",
    font=dict(color="#E6E6E6", size=14),
    width=1400,
    height=700,
    yaxis=dict(
        title="%",
        exponentformat="none",
        showgrid=False,
        zeroline=False
    ),
    xaxis=dict(
        showgrid=False,
        zeroline=False
    ),
    hovermode="x unified",
)

fig.show()


- **CPIAUCSL data plot**

In [11]:
fig = px.line(
    CPIAUCSL_df,
    x=CPIAUCSL_df.index,
    y="CPIAUCSL",
    markers=True
)

fig.update_layout(
    title=dict(text='Consumer Price Index for All Urban Consumers: All Items in U.S. City Average'),
    width=1000,
    height=600,
    yaxis=dict(title="CPIAUCSL"),
    hovermode="x unified",
    template="plotly_dark"
)

fig.show()